In [47]:
import spacy
import json
import os
from pathlib import Path
import random
from spacy.training import Example
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [43]:
DATA_DIR = os.path.join("..", "data", "annotated")
print(os.listdir(DATA_DIR))

['instruction.md', 'ML', 'Python', 'tags.json']


In [46]:
def load_annotated_data(base_dir):
    data = []
    folders = ["Python", "ML"]  # subfolders to load

    for folder in folders:
        folder_path = os.path.join(base_dir, folder)
        if not os.path.exists(folder_path):
            print(f"⚠️ Folder not found: {folder_path}")
            continue

        for root, _, files in os.walk(folder_path):
            for filename in files:
                if filename.endswith(".json"):
                    file_path = os.path.join(root, filename)
                    with open(file_path, "r", encoding="utf-8") as f:
                        item = json.load(f)

                        # Iterate through annotated texts
                        for text, ann in item["annotations"]:
                            entities = [
                                (start, end, label)
                                for start, end, label in ann.get("entities", [])
                            ]
                            if text.strip():
                                data.append((text, {"entities": entities}))

    print(f"Loaded {len(data)} annotated samples from {base_dir}")
    return data

data = load_annotated_data(DATA_DIR)

Loaded 74 annotated samples from ..\data\annotated


In [48]:
tags_path = os.path.join(DATA_DIR, "tags.json")
with open(tags_path, "r", encoding="utf-8") as f:
    tags = json.load(f)

labels = [t["name"] for t in tags]
print(f"Loaded {len(labels)} entity labels from tags.json:\n{labels}")

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Train: {len(train_data)} | Test: {len(test_data)}")

nlp = spacy.blank("en") 
for pipe_name in nlp.pipe_names:
    nlp.remove_pipe(pipe_name)

ner = nlp.add_pipe("ner")

for label in labels:
    ner.add_label(label)

Loaded 10 entity labels from tags.json:
['SKILL_HARD', 'SKILL_SOFT', 'ENGLISH_LEVEL', 'DEGREE', 'EXPERIENCE_LEVEL', 'EXPERIENCE_YEARS', 'BENEFIT', 'LOCATION', 'COMPANY', 'ROLE']
Train: 59 | Test: 15


In [52]:
optimizer = nlp.initialize()
n_iter = 100

for i in range(n_iter):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in tqdm(train_data, desc=f"Epoch {i+1}/{n_iter}"):
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.2, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i+1}: {losses}")

correct, total = 0, 0
for text, ann in test_data:
    doc = nlp(text)
    pred = {(e.start_char, e.end_char, e.label_) for e in doc.ents}
    true = set(tuple(e) for e in ann["entities"])
    correct += len(pred & true)
    total += len(true)

Epoch 1/100: 100%|██████████| 59/59 [00:07<00:00,  8.15it/s]


Losses at iteration 1: {'ner': 8189.255520219892}


Epoch 2/100: 100%|██████████| 59/59 [00:07<00:00,  8.35it/s]


Losses at iteration 2: {'ner': 3402.0473010801834}


Epoch 3/100: 100%|██████████| 59/59 [00:07<00:00,  8.09it/s]


...

Losses at iteration 98: {'ner': 355.75479916283894}


Epoch 99/100: 100%|██████████| 59/59 [00:06<00:00,  8.61it/s]


Losses at iteration 99: {'ner': 339.0287944233665}


Epoch 100/100: 100%|██████████| 59/59 [00:06<00:00,  8.56it/s]


Losses at iteration 100: {'ner': 339.607340809433}


In [53]:
precision = correct / total if total > 0 else 0
print(f"Baseline precision: {precision:.3f}")

Baseline precision: 0.576


In [54]:
example_text = random.choice(test_data)[0]
doc = nlp(example_text)
print("\n🔍 Example predictions:")
for ent in doc.ents:
    print(f"{ent.text:<30} → {ent.label_}")


🔍 Example predictions:
Full Stack Developer           → ROLE
Python+Angular                 → SKILL_HARD
4+ years of experience         → EXPERIENCE_YEARS
Python                         → SKILL_HARD
Django                         → SKILL_HARD
3+ years of experience         → EXPERIENCE_YEARS
PostgreSQL                     → SKILL_HARD
RDBMS                          → SKILL_HARD
SQL                            → SKILL_HARD
AWS                            → SKILL_HARD
Azure                          → SKILL_HARD
REST API                       → SKILL_HARD
Aiohttp                        → SKILL_HARD
FastAPI                        → SKILL_HARD
4+ years of experience         → EXPERIENCE_YEARS
Angular.                       → SKILL_HARD
HTML                           → SKILL_HARD
CSS                            → SKILL_HARD
SCSS                           → SKILL_HARD
JavaScript                     → SKILL_HARD
TypeScript.                    → SKILL_HARD
REST                           → SKILL_H

In [ ]:
examples = []
for text, annotations in test_data:
    doc = nlp.make_doc(text)
    examples.append(Example.from_dict(doc, annotations))

scorer = nlp.evaluate(examples)

print("\n📊 Evaluation results:")
print(f"Precision: {scorer['ents_p']:.3f}")
print(f"Recall:    {scorer['ents_r']:.3f}")
print(f"F-score:   {scorer['ents_f']:.3f}")
print(f"Entities per type:\n{scorer['ents_per_type']}")


📊 Evaluation results:
Precision: 0.676
Recall:    0.630
F-score:   0.652
Entities per type:
{'ROLE': {'p': 0.48, 'r': 0.631578947368421, 'f': 0.5454545454545454}, 'SKILL_HARD': {'p': 0.7772020725388601, 'r': 0.6912442396313364, 'f': 0.7317073170731708}, 'EXPERIENCE_YEARS': {'p': 0.6923076923076923, 'r': 0.6428571428571429, 'f': 0.6666666666666666}, 'SKILL_SOFT': {'p': 0.3333333333333333, 'r': 0.3125, 'f': 0.3225806451612903}, 'BENEFIT': {'p': 0.5573770491803278, 'r': 0.576271186440678, 'f': 0.5666666666666667}, 'EXPERIENCE_LEVEL': {'p': 1.0, 'r': 0.7777777777777778, 'f': 0.8750000000000001}, 'ENGLISH_LEVEL': {'p': 0.6666666666666666, 'r': 0.5, 'f': 0.5714285714285715}, 'LOCATION': {'p': 0.2, 'r': 0.5, 'f': 0.28571428571428575}, 'COMPANY': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DEGREE': {'p': 0.0, 'r': 0.0, 'f': 0.0}}
